In [1]:
pip install pyspark

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 44.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=1dd6ce47cd8e5af8c1fd014fbaa68f185a2e77082815fc4d51d89e5ed3701843
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [4]:

# sc.stop()
import sys

from pyspark import SparkContext, SparkConf
from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser

def subsets(arr):

    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupportRDD(itemSetRDD, broadcastTransactionList, minSupportBroadcast, freqSetRDD):
    
    # print("returnItemsWithMinSupportRDD")
    # print("broadcastTransactionList",broadcastTransactionList)
    # print("itemSetRDD.collect()  returnItemsWithMinSupportRDD: ",itemSetRDD.collect())

    def issubsetRDD(currentSet,broadcastTransactionList):
      listItem = []
      for transaction in broadcastTransactionList:
        if currentSet.issubset(transaction):
          listItem.append((currentSet, 1))
      
      return listItem
    # print("BEFORE itemSetRDD.collect(): ",itemSetRDD.collect())
    itemSetRDD = itemSetRDD.flatMap(lambda currentSet : issubsetRDD(currentSet,broadcastTransactionList)).reduceByKey(lambda a,b: a+b).distinct()
    # .filter(lambda currentSet : issubsetRDD(currentSet,broadcastTransactionList)).map(lambda item: (item, 1))
    # print("AFTER itemSetRDD itemSetRDD.collect(): ",itemSetRDD.collect())
    # .map(lambda item: (item, 1)).reduceByKey(lambda a,b: a+b).distinct() .reduceByKey(lambda a,b: a+b).distinct()
    # iemSetRDD looks like [( (a, b, c), 1), ((e, g, f), 5) ...] After
    # .map(lambda item: (item, 1)).reduceByKey(lambda a,b: a+b)
    # freqSetRDD looks like [( (a, b, c), 1), ((e, g, f), 5) ...]

    # print("AFTER itemSetRDD.collect(): ",itemSetRDD.collect())
    freqSetRDD = freqSetRDD.union(itemSetRDD)

    localSetRDD = itemSetRDD
    _itemSetRDD = localSetRDD.filter(lambda count : count[1] >=minSupportBroadcast).map(lambda word: (word[0])) 

    # _itemSetRDD look like [(a , b, c), (b, d ,e) ...]
    return _itemSetRDD, freqSetRDD

def joinSetRDD(sc, itemSetRDD, length):
    
    broadcastLength = sc.broadcast(length).value
    # print("broadcastLength",broadcastLength)
    # print("itemSetRDD.collect()",itemSetRDD.collect())
    # returnRDD1 = itemSetRDD.(itemSetRDD)
    # print("returnRDD1.collect()",returnRDD1.collect()) 
    returnRDD = itemSetRDD.cartesian(itemSetRDD).map(lambda x : x[0].union(x[1])).filter(lambda x: len(x) == broadcastLength).distinct()
    # print("returnRDD.collect() joinSetRDD", returnRDD.collect())
    return returnRDD


def runApriori(sc, file1, minSupport, minConfidence):
   
    fileRDD = sc.textFile(file1)
    # fileRDD = sc.parallelize(file1)
    transactionListRDD = fileRDD.map(lambda line: line.split(" ")).map(frozenset)
    # print("transactionListRDD.collect()",transactionListRDD.collect())
    # itemSetRDD = transactionListRDD.map(lambda line: list(line)).map(frozenset).distinct()
    itemSetRDD = fileRDD.flatMap(lambda line: line.split(" ")).map(lambda x: frozenset((x,))).distinct()
    minSupportBroadcast = sc.broadcast(minSupport).value
    # print("itemSetRDD.collect()",itemSetRDD.collect())
    broadcastTransactionListRDD = sc.broadcast(transactionListRDD.collect()).value
    # itemSet, transactionList = getItemSetTransactionList(data_iter)
    
    freqSetRDD = sc.parallelize([])
    largeSetRDD = sc.parallelize([])
   
    # print(freqSetRDD.collect())
    assocRules = dict()

    # oneCSetRDD look like [(a , b, c), (b, d ,e) ...]
    oneCSetRDD, freqSetRDD = returnItemsWithMinSupportRDD(itemSetRDD, broadcastTransactionListRDD, minSupportBroadcast, freqSetRDD)
    
    currentLSetRDD = oneCSetRDD
    # print(currentLSetRDD.collect())

    k = 2

    while(currentLSetRDD.collect() != []):
        # print("k: ",k)
        kLengthBroadcast = sc.broadcast(k).value
        # print("largeSetRDD", largeSetRDD.collect())
        largeSetRDD = largeSetRDD.union(currentLSetRDD.map(lambda x: (kLengthBroadcast-1,x)))
        # print("largeSetRDD",largeSetRDD.collect())
        # bleow is for currentLSet = joinSet(currentLSet, k)
        
        #This joinSetRDD make something like [(a,b), (a,c)] from [(a),(b), ...]
        currentLSetRDD = joinSetRDD(sc, currentLSetRDD, k)
        #Blow is currentCSet = returnItemsWithMinSupport(currentLSet,
        #                                         transactionList,
        #                                         minSupport,
        #                                         freqSet)
        # currentCSet look like [(a , b, c), (b, d ,e) ...]
        # print("currentLSetRDD",currentLSetRDD.collect())
        currentCSetRDD, freqSetRDD = returnItemsWithMinSupportRDD(currentLSetRDD,  broadcastTransactionListRDD, minSupport, freqSetRDD)
        currentLSetRDD = currentCSetRDD
        # print("currentLSet",currentLSet.collect())
        k = k + 1

    def getSupportRDD(item, broadcastFreqSetRDD):
      
      for i in broadcastFreqSetRDD:
          if i[0] == item:
              return i[1]
        
        # broadcastFreqSetRDD = sc.broadcast(freqSetRDD.collect()).value   .map(lambda x : dict((x,)))
       
    # print(" BEFORE freqSetRDD",freqSetRDD.collect())  
    
    broadcastFreqSetRDD = sc.broadcast(freqSetRDD.collect()).value
    # broadcastFreqSetRDD [(frozenset({'1'}), 6), (frozenset({'2'}), 7),
    # print("broadcastFreqSetRDD",broadcastFreqSetRDD)
    # print("freqSetRDD",freqSetRDD.collect())  
    # toRetItems = []
    toRetItemsRDD = sc.parallelize([])
    # for key, value in largeSet.items():
    #     toRetItems.extend([(tuple(item), getSupport(item))
    #                        for item in value])
    # largeSetRDD look like [(1, frozenset({'1'})), (1, frozenset({'2'})), ...
    # print("largeSetRDD.collect()",largeSetRDD.collect())
    toRetItemsRDD = largeSetRDD.map(lambda x : x[1]).map(lambda x : (tuple(x), getSupportRDD(x, broadcastFreqSetRDD)))
    # print("toRetItemsRDD.collect()", toRetItemsRDD.collect())  
    # print("largeSetRDD.collect()",largeSetRDD.collect())

    
    # toRetRulesRDD = sc.parallelize([])
    # def subsets(arr):
    #   """ Returns non empty subsets of arr"""
    #   return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])
    # def getSupportRDD(item, broadcastFreqSetRDD):
    #       """local function which Returns the support of an item"""
    #       for i in broadcastFreqSetRDD:
    #           if i[0] == item:
    #               return i[1]
    

    def loopForRetRule(largeSet, broadcastFreqSetRDD):
          toRetRules = []
          # largeSetRDD look like [(1, frozenset({'1'})), (1, frozenset({'2'})), ...
          _subsets = map(frozenset, [x for x in subsets(largeSet)])
          for element in _subsets:
              remain = largeSet.difference(element)
              if len(remain) > 0:
                  confidence = getSupportRDD(largeSet, broadcastFreqSetRDD)/getSupportRDD(element, broadcastFreqSetRDD)
                  if confidence >= 0.1:
                      toRetRules.append(((tuple(element), tuple(remain)),
                                          confidence))
          return toRetRules
    # print("largeSetRDD.collect()",largeSetRDD.collect())
    # print("broadcastFreqSetRDD", broadcastFreqSetRDD)
    toRetRulesRDD = largeSetRDD.filter(lambda x : x[0] > 1).map(lambda x : x[1]).flatMap(lambda largeSet : loopForRetRule(largeSet, broadcastFreqSetRDD))
    # flatMap(lambda largeSet : loopForRetRule(largeSet, broadcastFreqSetRDD))
    # print("toRetRulesRDD.collect()", toRetRulesRDD.collect())  
    return toRetItemsRDD, toRetRulesRDD


def printResults(items, rules):
    print("==============================RESULT=============================")
    
    for item, support in sorted(items, key=lambda x: x[1]):
        print("item: %s , %.3f" % (str(item), support))
    print("\n------------------------ RULES:")
    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        print("Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence))

file1 = '/content/drive/My Drive/CS5683/browsing.txt'
sc = SparkContext("local","PySpark Word Count Exmaple")
items, rules = runApriori(sc,file1, 85, 0.9)


# print("items.collect()",items.collect())
# print("rules.collect()",rules.collect())
printResults(items.collect(), rules.collect())





Streaming output truncated to the last 5000 lines.
Rule: ('', 'GRO85051', 'DAI62779') ==> ('FRO40251', 'DAI75645') , 0.403
Rule: ('DAI85309', 'GRO94758') ==> ('DAI62779',) , 0.403
Rule: ('DAI85309', 'GRO94758') ==> ('', 'DAI62779') , 0.403
Rule: ('', 'DAI85309', 'GRO94758') ==> ('DAI62779',) , 0.403
Rule: ('ELE32164', 'DAI43223') ==> ('GRO59710',) , 0.404
Rule: ('ELE32164', 'DAI43223') ==> ('DAI62779',) , 0.404
Rule: ('ELE32164', 'DAI43223') ==> ('', 'GRO59710') , 0.404
Rule: ('', 'ELE32164', 'DAI43223') ==> ('GRO59710',) , 0.404
Rule: ('ELE32164', 'DAI43223') ==> ('', 'DAI62779') , 0.404
Rule: ('', 'ELE32164', 'DAI43223') ==> ('DAI62779',) , 0.404
Rule: ('FRO92469',) ==> ('FRO40251', 'SNA80324') , 0.404
Rule: ('FRO92469',) ==> ('', 'FRO40251', 'SNA80324') , 0.404
Rule: ('', 'FRO92469') ==> ('FRO40251', 'SNA80324') , 0.404
Rule: ('DAI62779', 'GRO81087') ==> ('ELE17451',) , 0.404
Rule: ('DAI62779', 'GRO81087') ==> ('', 'ELE17451') , 0.404
Rule: ('', 'DAI62779', 'GRO81087') ==> ('ELE1745

In [ ]:
sc.stop()